In [1]:
import os
import sys
sys.path.append(os.path.dirname(os.path.abspath(os.path.join('../'))))

In [2]:
import numpy
import torchvision

train = torchvision.datasets.MNIST('mnist', train = True, download = False)
test = torchvision.datasets.MNIST('mnist', train = False, download = False)

train_images = numpy.array([ numpy.array(item[0]) for item in train ])
train_labels = numpy.array([ item[1] for item in train ])

test_images = numpy.array([ numpy.array(item[0]) for item in test ])
test_labels = numpy.array([ item[1] for item in test ])

In [ ]:
import matplotlib.pyplot as plt
import gtda.images


b = gtda.images.Binarizer(threshold = 0.1)
df = gtda.images.DensityFiltration(n_jobs = -1, radius = 3, metric = 'chebyshev')

img = df.fit_transform(b.fit_transform(test_images[:10]))[4]
plt.imshow(img)

In [ ]:
import matplotlib.pyplot as plt
import gtda.images


i = gtda.images.Inverter()
df = gtda.images.DensityFiltration(n_jobs = -1, radius = 3, metric = 'chebyshev')

img = df.fit_transform(b.fit_transform(test_images[:10]))[4]

img = i.fit_transform(test_images[:10])[4]
plt.imshow(img)

In [ ]:
import gtda.homology

gtda.homology.CubicalPersistence().fit_transform_plot(
    i.fit_transform(test_images[7:10])
)

In [ ]:
import gtda.homology

gtda.homology.CubicalPersistence().fit_transform_plot(
    test_images[7:10]
)

In [ ]:
_, width, height = train_images.shape

num_centers = 5

w_step = (width - num_centers) // (num_centers + 1)
numpy.arange(w_step, width, w_step + 1)

In [5]:
import itertools

import cvtda.topology

f = cvtda.topology.GreyscaleToFiltrations(
    radial_filtration_centers=list(itertools.product([ 3, 8, 13, 18, 23 ], [ 3, 8, 13, 18, 23 ]))
).fit_transform(train_images[:100])

Fitting the binarizer


Fitting the filtrations: 100%|██████████| 38/38 [00:00<00:00, 3588.91it/s]


Fitting the inverter


Fitting the inverted filtrations: 100%|██████████| 38/38 [00:00<00:00, 4197.07it/s]


Fitting complete


Inverted filtrations: 100%|██████████| 38/38 [00:00<00:00, 74.41it/s]


In [6]:
res = cvtda.topology.GreyscaleToPointClouds().fit_transform(train_images)

len(res[0])

Fitting the binarizer
Fitting the inverter
Fitting the converter
Fitting image_to_point_cloud
Fitting complete
Applying the binarizer
Applying the inverter


Converting inverted images to point clouds: 100%|██████████| 60000/60000 [00:01<00:00, 45678.75it/s]


Converting binary to point clouds


Combining the point clouds:   0%|          | 0/60000 [00:00<?, ?it/s]

4

In [6]:
import cvtda.utils


fb = cvtda.utils.FlattenBatch().fit(f)
print(f.shape)

g = fb.transform(f)
print(g.shape)

e = fb.inverse_transform(g)
print(e.shape)

numpy.allclose(f, e)

(100, 79, 28, 28)
(7900, 28, 28)
(100, 79, 28, 28)


True

In [15]:
import itertools

a = [
    [ [ 1, 2 ], [ 3 ] ],
    [ [ 4 ], [ 5, 6, 7 ] ]
]
list(itertools.chain(*a))

[[1, 2], [3], [4], [5, 6, 7]]

In [28]:
import typing

import numpy
import sklearn.base

T = typing.TypeVar("T")

class FlattenBatch(sklearn.base.TransformerMixin):
    def fit(self, data: typing.Iterable[typing.Iterable[T]]):
        self.dimension_ = len(data[0])
        self.fitted_ = True
        return self
    
    def transform(self, data: typing.Iterable[typing.Iterable[T]]) -> typing.Iterable[T]:
        assert self.fitted_ is True
        assert len(data[0]) == self.dimension_

        if type(data) == numpy.ndarray:
            return data.reshape((-1, *data.shape[2:]))
        else:
            return list(itertools.chain(*data))
    
    def inverse_transform(self, data: typing.Iterable[T]) -> typing.Iterable[typing.Iterable[T]]:
        assert self.fitted_ is True
        assert len(data) % self.dimension_ == 0

        if type(data) == numpy.ndarray:
            return data.reshape((-1, self.dimension_, *data.shape[1:]))
        else:
            return [
                data[i:i + self.dimension_]
                for i in range(0, len(data), self.dimension_)
            ]


In [38]:
import cvtda.utils


fb = FlattenBatch().fit(res)

print(len(res))
a = fb.transform(res)
print(len(a))

b = fb.inverse_transform(a)
print(len(b))

for i in range(60000):
    for j in range(4):
        if not numpy.allclose(res[i][j], b[i][j]):
            print(i, j)


60000
240000
60000


In [42]:
fb = FlattenBatch().fit(f)

g = fb.transform(f)
print(g.shape)

e = fb.inverse_transform(g)
print(e.shape)

numpy.allclose(f, e)

(7900, 28, 28)
(100, 79, 28, 28)


True

In [1]:
import cvtda.utils